In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import sys
import matplotlib.pyplot as plt
import seaborn as sns

# In this first section, we build a simple recommender that recommends food based on how much people have apprciated it

In [2]:
df = pd.read_csv('/kaggle/input/food-com-recipes-and-user-interactions/RAW_recipes.csv')

In [3]:
df2=pd.read_csv('/kaggle/input/food-com-recipes-and-user-interactions/RAW_interactions.csv')

In [4]:
df.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


In [5]:
df2.head()
df2=df2.drop(['user_id','date','review'],axis=1)

In [6]:
df2.head()

,recipe_id,rating
0,40893,4
1,40893,5
2,44394,4
3,85009,5
4,85009,5


In [7]:
#Store average rating of meals
C=df2['rating'].mean()

In [8]:
#Group seperate meal ratings together and get average meal score equally prevent recipe_id from being an index
df3=df2.groupby(['recipe_id'])['rating'].agg(['mean','count'])
df3.reset_index(inplace=True)

In [9]:
#Rename columns 
df3.rename(columns={'mean':'average_rating','count':'votes'},inplace=True)

In [10]:
#Merge ratings dataframe with recipe Dataframe
food_df = pd.merge(df, df3, left_on='id',right_on='recipe_id')

In [11]:
#Show the number of votes possesed by the top 10% meals, based on number of votes
q=df3['votes'].quantile(0.9)
print(q)

9.0


In [12]:
#Get meals which have received a substantial number of votes
top_meals=food_df.copy().loc[df3['votes']>=q]

In [13]:
#This function permits us to compute a special metric which takes into account the number of people
#that have voted for a meal and not just it's average rating
def weighted_rating(x, m=q, c=C):
    v = x['votes']
    R = x['average_rating']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [14]:
#Select top meals from Dataframe
top_meals['Score']=top_meals.apply(weighted_rating, axis=1)
food_df['Score']=food_df.apply(weighted_rating, axis=1)

In [15]:
#Sort best meals based on their scores
top_meals=top_meals.sort_values('Score',ascending=False)

#Print the top 20 meals
top_meals[['name', 'votes', 'average_rating', 'Score']].head(20)

,name,votes,average_rating,Score
14346,baked ham glazed with pineapple and chipotle p...,27,5.000000,4.852754
51504,chocolate raspberry or strawberry tall cake,37,4.945946,4.841286
177694,rr s caramelized onions,30,4.966667,4.838440
134161,mexican coffee caf mexicano,36,4.944444,4.837759
128474,magic white sauce and variations,23,5.000000,4.834348
192238,southeastern north carolina pulled pork,22,5.000000,4.829005
173099,reuben sandwich our way,20,5.000000,4.817212
83175,feta marinated,20,5.000000,4.817212
2165,a 1 cherry cobbler tart a1,20,5.000000,4.817212
224503,warm roasted root vegetable and chicken salad ...,47,4.893617,4.816056


# Complex Recommendation

 In this section, we will build a more complex recommender that recommends meals based on how much they are appreciated and how similar they are in general(*cookery time, ingredients,nutritional value, etc.)* This sort of system is ideal for a cook who wishes to please guests by cooking meals similar to what they appreciate without having to stress his cooking skills

In [16]:
food_df.columns

Index(['name', 'id', 'minutes', 'contributor_id', 'submitted', 'tags',
       'nutrition', 'n_steps', 'steps', 'description', 'ingredients',
       'n_ingredients', 'recipe_id', 'average_rating', 'votes', 'Score'],
      dtype='object')

In [17]:
food_df.loc[0]['nutrition']

'[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]'

In [18]:
#We import literal eval to convert objects into corresponding python objects
from ast import literal_eval

In [19]:
features=['steps','ingredients']
for feature in features:
    food_df[feature] = food_df[feature].apply(literal_eval)